In [1]:
#hide
#default_exp dev.clientgen

In [2]:
#exports
from jinja2 import Template

from ElexonDataPortal.dev import specgen, rawgen, clientprep

In [3]:
import os
from dotenv import load_dotenv

assert load_dotenv('../.env'), 'Environment variables could not be loaded'

api_key = os.environ['BMRS_API_KEY']

<br>

### API Client Generation

In [4]:
#exports
def generate_streams(API_yaml_fp):
    request_type_to_date_range_example = {
        'SP_and_date': ('2020-01-01', '2020-01-01 1:30'),
        'date_range': ('2020-01-01', '2020-01-07'),
        'date_time_range': ('2020-01-01', '2020-01-07'),
        'year': ('2019-01-01', '2021-01-01'),
        'year_and_month': ('2020-01-01', '2020-06-01'),
        'year_and_week': ('2020-01-01', '2020-06-01'),
        'non_temporal': (None, None)
    }
    
    API_yaml = specgen.load_API_yaml(API_yaml_fp)
    functions = rawgen.construct_all_functions(API_yaml)
    method_info = clientprep.construct_method_info_dict(API_yaml_fp)

    streams = list()

    for function in functions:
        name = function['name']
        function_method_info = method_info[name]

        stream = dict()
        stream['name'] = name
        stream['description'] = function['description']
        stream['date_range_example'] = request_type_to_date_range_example[function_method_info['request_type']]
        stream['extra_kwargs'] = [param for param in function['parameters'] if param['name'] not in list(function_method_info['kwargs_map'].values())+['APIKey', 'ServiceType']]
        stream['request_type'] = function_method_info['request_type']
        stream['kwargs_map'] = function_method_info['kwargs_map']
        stream['func_params'] = list(function_method_info['func_kwargs'].keys())

        streams += [stream]
        
    return streams

In [5]:
API_yaml_fp = '../data/BMRS_API.yaml'

streams = generate_streams(API_yaml_fp)

streams[0]

{'name': 'get_B0610',
 'description': 'Actual Total Load per Bidding Zone',
 'date_range_example': ('2020-01-01', '2020-01-01 1:30'),
 'extra_kwargs': [],
 'request_type': 'SP_and_date',
 'kwargs_map': {'date': 'SettlementDate', 'SP': 'Period'},
 'func_params': ['APIKey', 'date', 'SP', 'ServiceType']}

In [6]:
#exports
def save_api_client(
    API_yaml_fp: str,
    in_fp: str='../templates/api.py',
    out_fp: str='../ElexonDataPortal/api.py'
):
    streams = generate_streams(API_yaml_fp)
    rendered_schema = Template(open(in_fp).read()).render(streams=streams)

    with open(out_fp, 'w') as f:
        try:
            f.write(rendered_schema)
        except e as exc:
            raise exc

In [7]:
save_api_client(API_yaml_fp)

<br>

### API Client Testing

In [8]:
from ElexonDataPortal import api

client = api.Client(api_key)

df = client.get_WINDFORFUELHH()

df.head(3)

c:\users\ayrto\desktop\phd\data\bmrs\elexon-bmrs-api-wrapper\ElexonDataPortal\dev\orchestrator.py:115: UserWarning: Response was capped, request is rerunning for missing data from 2020-01-05
  warn(f'Response was capped, request is rerunning for missing data from {start_date}')


,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag
0,WIND,2020-01-01,1,2019-12-30 02:30:00,5396,2019-12-31 22:30:00,5001,2020-01-01 00:30:00,5004,Y
1,WIND,2020-01-01,2,NULL,NULL,NULL,NULL,2020-01-01 01:00:00,5043,Y
2,WIND,2020-01-01,3,2019-12-30 02:30:00,5440,2019-12-31 22:30:00,4857,2020-01-01 01:30:00,5030,Y


In [14]:
client_methods = [method for method in dir(client) if ('__' not in method) and (method not in ['api_key', 'methods', 'n_retry_attempts'])]
method_to_df = dict()

for client_method in client_methods:
    method_to_df[client_method] = getattr(client, client_method)()

PHYBMDATA: 100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.13s/it]


In [15]:
method = 'get_B1440'

print(getattr(client, method).__doc__)
method_to_df[method].head(3)


        Generation forecasts for Wind and Solar
        
        Parameters:
            start_date (str)
            end_date (str)
            ProcessType (str)
        


,local_datetime,timeSeriesID,businessType,powerSystemResourceType,settlementDate,processType,settlementPeriod,quantity,documentType,curveType,resolution,activeFlag,documentID,documentRevNum
0,2020-01-01 00:00:00+00:00,NGET-EMFIP-DGWS-TS-00034592,Solar generation,"""Solar""",2020-01-01,Day Ahead,1,0,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1
1,2020-01-01 00:00:00+00:00,NGET-EMFIP-DGWS-TS-00034590,Wind generation,"""Wind Offshore""",2020-01-01,Day Ahead,1,2843.181,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1
2,2020-01-01 00:00:00+00:00,NGET-EMFIP-DGWS-TS-00034591,Wind generation,"""Wind Onshore""",2020-01-01,Day Ahead,1,3024.238,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1


In [16]:
#hide
from ElexonDataPortal.dev.nbdev import notebook2script
notebook2script('05-orchestrator.ipynb')

Converted 05-orchestrator.ipynb.
